In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [8]:
db_host = '18.136.56.185'
username = 'dm_team2'
user_pass = 'dm_team1118#'
db_name = 'project_sales'

conn = create_engine('mysql+pymysql://'+username+':'+user_pass+'@'+db_host+'/'+db_name)
conn.table_names()

['data']

In [4]:
query = 'select * from data'

In [9]:
data = pd.read_sql(query,conn)

In [6]:
data.shape

(7422, 9)

In [10]:
data.isnull().sum()

Created          0
Product_ID       0
Source           0
Mobile           0
EMAIL            0
Sales_Agent      0
Location         0
Delivery_Mode    0
Status           0
dtype: int64

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7422 entries, 0 to 7421
Data columns (total 9 columns):
Created          7422 non-null object
Product_ID       7422 non-null object
Source           7422 non-null object
Mobile           7422 non-null object
EMAIL            7422 non-null object
Sales_Agent      7422 non-null object
Location         7422 non-null object
Delivery_Mode    7422 non-null object
Status           7422 non-null object
dtypes: object(9)
memory usage: 521.9+ KB


# Dropping columns which are not required


In [12]:
data.drop(['Created','Mobile','EMAIL'],axis=1,inplace=True)
data.head()

,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
0,,Website,Sales-Agent-11,,Mode-5,Open
1,,Website,Sales-Agent-10,,Mode-5,Open
2,,Website,Sales-Agent-10,,Mode-5,Open
3,,Website,Sales-Agent-10,,Mode-5,Open
4,,Website,Sales-Agent-10,,Mode-5,Open


In [13]:
data.replace('',np.nan,inplace=True)
data.dropna(inplace=True)
data.reset_index(inplace=True,drop=True)
data.shape

(7328, 6)

In [14]:
data.Source.replace(['Live Chat-Direct','Live Chat-Google Organic','Live Chat -PPC','Live Chat-Blog','Live Chat-Quora',
                    'Live Chat-CPC','Live Chat-Google Ads','Live Chat-Adwords Remarketing','Live Chat-Youtube',
                    'Live Chat-Justdial'],'Live Chat',inplace=True)
data.Source.replace(['Existing Client','CRM form','Personal Contact'],'Existing Customer',inplace=True)
data.Source.replace('By Recommendation','Customer Referral',inplace=True)
data.Source.replace(['US Website','Just Dial'],'Website',inplace=True)
data.Source.replace(['E-mail Campaign','SMS Campaign','E-Mail Message','Other'],'Campaign',inplace=True)

In [15]:
data.Location.replace(['UAE','USA','UK','AUSTRALIA','Singapore','Malaysia','EUROPE'],'Foreign',inplace=True)
data.Location.replace(['Howrah','Kolkata','Trivandrum'],'Other Locations',inplace=True)

In [16]:
data.Product_ID.replace(['8','6','3','17','22','11','7','28','0','23','24','16','13','4','14','26','12','2','21','25'],
                        'Less Sold Product',inplace=True)

In [17]:
data.Status.replace(['CONVERTED','converted','In Progress Positive','Potential','Long Term','Open'],'Good',inplace=True)
data.Status.replace(['LOST','In Progress Negative','Not Responding','Junk Lead','Just Enquiry'],'Bad',inplace=True)

In [18]:
data.shape

(7328, 6)

In [19]:
data.tail()

,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
7323,9,Call,Sales-Agent-6,Mumbai,Mode-4,Bad
7324,15,Call,Sales-Agent-12,Other Locations,Mode-5,Bad
7325,5,Live Chat,Sales-Agent-11,Bangalore,Mode-1,Bad
7326,Less Sold Product,Existing Customer,Sales-Agent-4,Other Locations,Mode-1,Bad
7327,Less Sold Product,Website,Sales-Agent-3,Chennai,Mode-1,Good


In [20]:
from sklearn.preprocessing import LabelEncoder

In [21]:
enc = LabelEncoder()
data.Product_ID = enc.fit_transform(data.Product_ID)
data.Source = enc.fit_transform(data.Source)
data.Sales_Agent = enc.fit_transform(data.Sales_Agent)
data.Location = enc.fit_transform(data.Location)
data.Delivery_Mode = enc.fit_transform(data.Delivery_Mode)
data.Status = enc.fit_transform(data.Status)
data.head()

,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
0,8,4,4,0,0,1
1,4,0,5,6,4,1
2,3,5,1,6,0,1
3,2,5,8,4,0,0
4,3,0,8,0,0,1


In [22]:
data.corr()

,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
Product_ID,1.000000,0.074868,0.056065,-0.226961,-0.181464,0.138943
Source,0.074868,1.000000,-0.023186,-0.003034,-0.216516,-0.015411
Sales_Agent,0.056065,-0.023186,1.000000,-0.140876,-0.224688,0.137074
Location,-0.226961,-0.003034,-0.140876,1.000000,0.414193,-0.347418
Delivery_Mode,-0.181464,-0.216516,-0.224688,0.414193,1.000000,-0.220445
Status,0.138943,-0.015411,0.137074,-0.347418,-0.220445,1.000000


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [24]:
X = data.iloc[:,:-1]
y = data.Status

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

## Logistic Regression

In [124]:
from sklearn.linear_model import LogisticRegression
model_logr = LogisticRegression()
model_logr.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [125]:
y_predict_log = model_logr.predict(X_test)

In [126]:
print(accuracy_score(y_test,y_predict_log))

0.6734879490677581


In [128]:
print(pd.crosstab(y_test,y_predict_log))

col_0      0    1
Status           
0       1030  289
1        429  451


## XGBoost Classifier


In [93]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [94]:
from xgboost import XGBClassifier
model_xgb = XGBClassifier(n_estimators=450,max_depth=17,gamma=5,learning_rate=0.01,random_state=10)
model_xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=5,
              learning_rate=0.01, max_delta_step=0, max_depth=17,
              min_child_weight=1, missing=None, n_estimators=450, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=10,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [95]:
y_predict_xgb = model_xgb.predict(X_test)

In [116]:
print(accuracy_score(y_test,y_predict_xgb))

0.7157798999545247


In [118]:
pd.crosstab(y_test,y_predict_xgb)

col_0,0,1
Status,,
0,1128,191
1,434,446


## RandomForestClassifier

In [76]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=10)

In [130]:
from sklearn.ensemble import RandomForestClassifier
classifier_rfg = RandomForestClassifier()
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
parameters = {'max_depth':[13,14,15,16,17],
              'random_state': range(0,4),
              'n_estimators':[105,110], 
             }
model_gridrf = GridSearchCV(model,parameters)
model_gridrf.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
           

In [132]:
y_predict_rf = model_gridrf.predict(X_test)

In [133]:
print(accuracy_score(y_test,y_predict_rf))

0.702137335152342


In [136]:
print(pd.crosstab(y_test,y_predict_rf))

col_0      0    1
Status           
0       1062  257
1        398  482
